In [6]:
import pandas as pd
import requests
import pprint
from config import api_key

In [7]:
us_file = pd.read_csv("Video_CSVs/USvideos.csv")
us_file.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [8]:
# see if there is any missing data
us_file.count()

video_id                  40949
trending_date             40949
title                     40949
channel_title             40949
category_id               40949
publish_time              40949
tags                      40949
views                     40949
likes                     40949
dislikes                  40949
comment_count             40949
thumbnail_link            40949
comments_disabled         40949
ratings_disabled          40949
video_error_or_removed    40949
description               40379
dtype: int64

In [9]:
# get api data to add category names
params = {"part":"snippet","regionCode":"US",
           "key":api_key}
base_url= "https://www.googleapis.com/youtube/v3/videoCategories"
response = requests.get(base_url, params=params).json()
response

{'kind': 'youtube#videoCategoryListResponse',
 'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'items': [{'kind': 'youtube#videoCategory',
   'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
   'id': '1',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Film & Animation',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
   'id': '2',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Autos & Vehicles',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
   'id': '10',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Music',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/HwXKamM1Q20q9BN-oBJavSGkfDI"',
   'id': '15',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdnt

In [11]:
id_list = []
title_list = []
for i in response['items']:
    #print(i)
    id_list.append(i['id'])
    title_list.append(i['snippet']['title'])

In [12]:
# Create DF
category_df=pd.DataFrame({"category_id":id_list,"title_list":title_list})
category_df["category_id"]=category_df["category_id"].astype("int64")
category_df.dtypes

category_id     int64
title_list     object
dtype: object

In [16]:
# Merge with us_df
us_df=pd.merge(us_file,category_df,on='category_id',how="left")
# focus on data that we are interested in
us_df=us_df.loc[:,("video_id", "trending_date", "title", "channel_title", "category_id", "views", "likes", "dislikes")]
# rename columns
us_df=us_df.rename(columns={"video_id":"Video ID", 
                            "trending_date": "Trending Date",
                            'title': "Title",
                            "channel_title":"Channel Title",
                            'category_id': "Category ID",
                            'views':'Views', 
                            "likes": 'Likes', 
                            'dislikes': "Dislikes",
                            'comment_count': 'Comment Count'})
us_df

,Video ID,Trending Date,Title,Channel Title,Category ID,Views,Likes,Dislikes
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,748374,57527,2966
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2418783,97185,6146
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,3191434,146033,5339
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,343168,10172,666
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2095731,132235,1989
...,...,...,...,...,...,...,...,...
40944,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,1685609,38160,1385
40945,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,1064798,60008,382
40946,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,1066451,48068,1032
40947,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,5660813,192957,2846


In [17]:
#sorted views in descending order (largest values at the top)
top_views=us_df.sort_values("Views",ascending=False)
#top 5 video views
top_5_views = top_views[:5]
top_5_views

,Video ID,Trending Date,Title,Channel Title,Category ID,Views,Likes,Dislikes
38547,VYOjWnS4cMY,18.02.06,Childish Gambino - This Is America (Official V...,ChildishGambinoVEVO,10,225211923,5023450,343541
38345,VYOjWnS4cMY,18.01.06,Childish Gambino - This Is America (Official V...,ChildishGambinoVEVO,10,220490543,4962403,338105
38146,VYOjWnS4cMY,18.31.05,Childish Gambino - This Is America (Official V...,ChildishGambinoVEVO,10,217750076,4934188,335462
37935,VYOjWnS4cMY,18.30.05,Childish Gambino - This Is America (Official V...,ChildishGambinoVEVO,10,210338856,4836448,326902
37730,VYOjWnS4cMY,18.29.05,Childish Gambino - This Is America (Official V...,ChildishGambinoVEVO,10,205643016,4776680,321493


In [18]:
#sorted views in ascending order (lowest values at the top)
bottom_views=us_df.sort_values("Views")
#bottom 5 video views
bottom_5_views = bottom_views[:5]
bottom_5_views

,Video ID,Trending Date,Title,Channel Title,Category ID,Views,Likes,Dislikes
14335,y6KYFcta4SE,18.26.01,"1 dead, others injured after Ky. school shooting",Newsy,25,549,9,0
14563,y6KYFcta4SE,18.27.01,"1 dead, others injured after Ky. school shooting",Newsy,25,554,9,0
14782,y6KYFcta4SE,18.28.01,"1 dead, others injured after Ky. school shooting",Newsy,25,559,9,0
14531,dQMZLXaa1L8,18.27.01,Artwork Forge,Palo Alto Online,17,658,1,0
546,-JVITToppE0,17.16.11,Coach Taggart Monday Presser Ahead of Arizona,GoDucksdotcom,17,687,10,2


In [19]:
#sort likes in descending order (largest values at the top)
top_likes=us_df.sort_values("Likes",ascending=False)
#top 5 video views
top_5_likes = top_likes[:5]
top_5_likes

,Video ID,Trending Date,Title,Channel Title,Category ID,Views,Likes,Dislikes
38273,7C2z4GqqS5E,18.01.06,BTS (방탄소년단) 'FAKE LOVE' Official MV,ibighit,10,123010920,5613827,206892
38072,7C2z4GqqS5E,18.31.05,BTS (방탄소년단) 'FAKE LOVE' Official MV,ibighit,10,121219886,5595203,205565
37861,7C2z4GqqS5E,18.30.05,BTS (방탄소년단) 'FAKE LOVE' Official MV,ibighit,10,115664850,5530568,200995
37655,7C2z4GqqS5E,18.29.05,BTS (방탄소년단) 'FAKE LOVE' Official MV,ibighit,10,111882133,5486349,197638
37453,7C2z4GqqS5E,18.28.05,BTS (방탄소년단) 'FAKE LOVE' Official MV,ibighit,10,107756615,5439015,194172


In [20]:
#sort likes in ascending order (lowest values at the top)
bottom_likes=us_df.sort_values("Likes")
#bottom 5 video views
bottom_5_likes = bottom_likes[:5]
bottom_5_likes

,Video ID,Trending Date,Title,Channel Title,Category ID,Views,Likes,Dislikes
1490,A_mlvG_nRsg,17.21.11,Kelly Oubre Punches John Wall in the Lead duri...,Rob Andretti,17,2197,0,2
14869,JWAvRIpuj1U,18.29.01,Vince McMahon's Major Sports Announcement,Alpha Entertainment,17,132093,0,0
1868,_akwfRuL4os,17.23.11,Highlights of the EA predatory behavior announ...,chris lee,22,160012,0,0
23516,_EMxJMU1dCo,18.13.03,GET READY WITH ME | Halo Beauty Launch Day,Tati,26,928082,0,0
16303,7Bq_H1ci9ec,18.05.02,The Unsung,MassMutual,27,18336,0,0


In [21]:
# Find top viewed Channels
channel_data = us_df.loc[:,("Channel Title", "Views", "Likes", "Dislikes")]
channel_data

,Channel Title,Views,Likes,Dislikes
0,CaseyNeistat,748374,57527,2966
1,LastWeekTonight,2418783,97185,6146
2,Rudy Mancuso,3191434,146033,5339
3,Good Mythical Morning,343168,10172,666
4,nigahiga,2095731,132235,1989
...,...,...,...,...
40944,AaronsAnimals,1685609,38160,1385
40945,zefrank1,1064798,60008,382
40946,Brad Mondo,1066451,48068,1032
40947,How It Should Have Ended,5660813,192957,2846


In [24]:
# sum the views, likes, and dislikes for each channel
topviewedchannel_us = channel_data.groupby("Channel Title").sum()["Views"]
toplikedchannel_us = channel_data.groupby("Channel Title").sum()["Likes"]
topdislikedchannel_us = channel_data.groupby("Channel Title").sum()["Dislikes"]

# create a dataframe of the total views, likes, and dislilkes of each channel
channel_summary = pd.DataFrame({"Total Views":topviewedchannel_us,
                               "Total Likes":toplikedchannel_us,
                               "Total Dislikes":topdislikedchannel_us})

# print dataframe
channel_summary

,Total Views,Total Likes,Total Dislikes
Channel Title,,,
12 News,177970,352,90
1MILLION Dance Studio,20959169,1399898,15030
1theK (원더케이),57375949,3663362,63329
20th Century Fox,1082872611,24419452,488761
2CELLOS,432186,22900,245
...,...,...,...
ワーナー ブラザース 公式チャンネル,7389323,154962,6128
圧倒的不審者の極み!,11417717,157447,12315
杰威爾音樂 JVR Music,400530463,3400678,229439


In [29]:
#sort channel views in descending order (largest values at the top)
topchannelviews_us = channel_summary.sort_values("Total Views", ascending=False)
#top 5 viewed channels
top_5_viewedchannels_us = topchannelviews_us[:5]
top_5_viewedchannels_us

,Total Views,Total Likes,Total Dislikes
Channel Title,,,
ChildishGambinoVEVO,3758488765,96700818,6054434
ibighit,2235906679,199247121,3467306
Dude Perfect,1870085178,60275557,1501477
Marvel Entertainment,1808998971,55873344,1031250
ArianaGrandeVevo,1576959172,52170970,1931230


In [30]:
#sort channel views in ascending order (lowest values at the top)
lowchannelviews_us = channel_summary.sort_values("Total Views")
#bottom 5 viewed channels
bottom_5_viewedchannels_us = lowchannelviews_us[:5]
bottom_5_viewedchannels_us

,Total Views,Total Likes,Total Dislikes
Channel Title,,,
90s Commercials,773,2,0
GoDucksdotcom,1391,20,4
OjO Commuter Scooter by OjO Electric,1706,6,1
Total Combat,1722,17,3
CHauserable,1807,6,3


In [31]:
#sort channel likes in descending order (largest values at the top)
topchannellikes_us = channel_summary.sort_values("Total Likes", ascending=False)
#top 5 liked channels
top_5_likedchannels_us = topchannellikes_us[:5]
top_5_likedchannels_us

,Total Views,Total Likes,Total Dislikes
Channel Title,,,
ibighit,2235906679,199247121,3467306
ChildishGambinoVEVO,3758488765,96700818,6054434
Dude Perfect,1870085178,60275557,1501477
Marvel Entertainment,1808998971,55873344,1031250
ArianaGrandeVevo,1576959172,52170970,1931230


In [32]:
#sort channel dislikes in descending order (largest values at the top)
topchanneldislikes_us = channel_summary.sort_values("Total Dislikes", ascending=False)
#top 5 disliked channels
top_5_dislikedchannels_us = topchanneldislikes_us[:5]
top_5_dislikedchannels_us

,Total Views,Total Likes,Total Dislikes
Channel Title,,,
Logan Paul Vlogs,484356303,31545290,13847251
YouTube Spotlight,791388476,20173324,10924092
ChildishGambinoVEVO,3758488765,96700818,6054434
Call of Duty,315404711,11553594,5644083
ibighit,2235906679,199247121,3467306
